# Техническая часть

In [1]:
!pip install -q sentencepiece
import sentencepiece
!pip install -q transformers

!pip install -q comet_ml
import comet_ml
!pip install -q pytorch-lightning

!pip -q install datasets
!pip -q install rouge_score

!git clone -q https://github.com/DanilDmitriev1999/QA

     |████████████████████████████████| 1.2MB 8.7MB/s 
     |████████████████████████████████| 2.1MB 8.5MB/s 
     |████████████████████████████████| 3.3MB 38.5MB/s 
     |████████████████████████████████| 901kB 52.4MB/s 
     |████████████████████████████████| 266kB 8.3MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 522kB 14.0MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 808kB 10.5MB/s 
     |████████████████████████████████| 645kB 21.7MB/s 
     |████████████████████████████████| 276kB 52.0MB/s 
     |████████████████████████████████| 112kB 33.8MB/s 
     |████████████████████████████████| 829kB 53.5MB/s 
     |████████████████████████████████| 1.3MB 50.1MB/s 
     |████████████████████████████████| 296kB 54.3MB/s 
     |████████████████████████████████| 143kB 53.6MB/s 
     |████████████████████████████████| 225kB 8.8MB/s 
     |████████████████████████████████| 245kB 40.2MB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')
import comet_ml

import numpy as np
import collections
import functools
import json
import random
import os
import math
import re

from io import open
from tqdm import tqdm
from pprint import pprint
from typing import List
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from transformers import (
    AdamW, MT5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup)

from datasets import load_metric

import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer, seed_everything

from QA.DataModule.dataset import *
from QA.DataModule.reader import *

from QA.model.BERT import *
from QA.utils.trainer import *

seed_everything(294)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    from torch.cuda import LongTensor
else:
    from torch import LongTensor
print(device)

Global seed set to 294


cuda


# Данные

In [3]:
train_file_path = '/content/QA/data/sber_squad/train-v1.1.json'
dev_file_path = '/content/QA/data/sber_squad/dev-v1.1.json'
train = ReadData(train_file_path)
train_data = train.data
dev = ReadData(dev_file_path)
dev_data = dev.data

tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

In [4]:
train_data[0]

{'context': 'В допетровское время искусство в России имело своим призванием служить исключительно религиозным целям, а так как православная церковь гнушается изваяниями человеческих фигур, то скульптура, в настоящем смысле слова, не могла в древней Руси не только развиваться, но и существовать. Правда, в некоторых местах, в особенности в бывших новгородских областях, пользовались уважением резные и раскрашенные изображения святых, но они были чужды всякого художественного значения и составляли изделия, возникшие под влиянием Запада. Собственно же на Руси проявления пластики ограничивались литьем небольших крестов, образов-складней, выбиванием окладов на образа и резьбою фигурных иконостасов. В числе плодов западно-европейской цивилизации Пётр Великий перенес в него и скульптуру, которая, однако, при этом государе и долго после него находилось здесь в руках приезжих иностранцев. Главным деятелем по части скульптуры в царствование Петра Великого и Анны Иоанновны был К. Б. Растрелли, отец

In [5]:
tokenizer.special_tokens_map

{'eos_token': '</s>', 'pad_token': '<pad>', 'unk_token': '<unk>'}

In [6]:
tokenizer('<s>')

{'input_ids': [1042, 263, 669, 1], 'attention_mask': [1, 1, 1, 1]}

In [4]:
class QGDataset:
    def __init__(self, dataset: List[dict], tokenizer) -> None:
        self.dataset = dataset
        self.tokenizer_context = lambda x: tokenizer.encode_plus(x,
                                                         add_special_tokens=True,
                                                         padding='max_length',
                                                         max_length=350,
                                                         truncation=True,
                                                         return_tensors="pt")
        self.tokenizer_question = lambda x: tokenizer.encode_plus(x,
                                                         add_special_tokens=True,
                                                         padding='max_length',
                                                         max_length=200,
                                                         truncation=True,
                                                         return_tensors="pt")

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx) -> dict:
        context = self.dataset[idx]['context'][0:150]
        question = self.dataset[idx]['qas'][0]['question'][0:100]
        encodings_context = self.tokenizer_context(context)
        encoding_question = self.tokenizer_question(question)

        result = {
            'input_ids': encodings_context['input_ids'].flatten(),
            'target_ids': encoding_question['input_ids'].flatten(),
            'input_attention_mask': encodings_context['attention_mask'].flatten(),
            'target_attention_mask': encoding_question['attention_mask'].flatten(),
        }
        # print(result)
        return result

        

In [5]:
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors='pt')

train_dataset = QGDataset(train_data, tokenizer)
train_iter = DataLoader(dataset=QGDataset(train_data, tokenizer),
                        batch_size=4)
dev_iter = DataLoader(dataset=QGDataset(dev_data, tokenizer),
                        batch_size=8)

In [10]:
for _ in train_iter:
    pass

In [ ]:
next(iter(train_iter))

# Model

In [6]:
import time


class QGmT5model(pl.LightningModule):
    def __init__(self, lr):
        super().__init__()
        self.lr = lr
        self.model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

        self.tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')
        self.rouge = load_metric('rouge')

        self.save_hyperparameters()

    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
    
    def forward(self, input_ids, input_attention_mask, 
                target_attention_mask=None, target_ids=None,):
        
        result = self.model(
            input_ids=input_ids,
            attention_mask=input_attention_mask,
            labels=target_ids)
        
        return result
    
    def lmap(self, f, x):
        return list(map(f, x))
    

    def _step(self, batch):
        lm_labels = batch['target_ids']
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        # print(f'input_ids: {batch["input_ids"].shape}')
        # print(f'input_attention_mask: {batch["input_attention_mask"].shape}')

        # print(f'target_idx: {batch["target_ids"].shape}')
        # print(f'target_attention_mask: {batch["target_attention_mask"].shape}')

        outputs = self(
            input_ids=batch["input_ids"],
            input_attention_mask=batch["input_attention_mask"],
            target_ids=batch["target_ids"],
            target_attention_mask=batch['target_attention_mask']
        )
    
        loss = outputs[0]
        return loss
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    def _generative_step(self, batch) :
        
        t0 = time.time()
        
        generated_ids = self.model.generate(
            batch["input_ids"],
            attention_mask=batch["input_attention_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_attention_mask'],
            max_length=100, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
        )
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"])
            
        gen_time = (time.time() - t0) / batch["input_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        self.log_dict({'val_loss': loss})

        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
        print(preds)
        print(target)
        self.rouge.add_batch(predictions=preds, references=target)
        
#         rouge_results = self.rouge_metric.compute() 
#         rouge_dict = self.parse_score(rouge_results)
#         base_metrics.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])
        
        return base_metrics
    
    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        logs = {"train_loss": loss}

        self.log_dict(logs)

        return loss
    
    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
    
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        logs = {"val_loss": avg_loss}
        
        rouge_results = self.rouge.compute() 
        rouge_dict = self.parse_score(rouge_results)
    
        logs.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])

        self.log_dict(logs)
        
        ## Clear out the lists for next epoch
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "rouge1" : rouge_results['rouge1'],
                "rougeL" : rouge_results['rougeL']}
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.lr, eps=1e-8)
        return optimizer

In [7]:
comet_logger = CometLogger(
    api_key="HWfJT3eyByVJWe4nEbi1pGosA",
    workspace="danildmitriev1999",
    project_name="qa",
    experiment_name="mT5 QG",
)

CometLogger will be initialized in online mode


In [8]:
lr = 3e-4

N_EPOCHS = 1
CLIP = 1.5

model = QGmT5model(lr).to(device)

trainer = Trainer(max_epochs=N_EPOCHS,
                  gpus=1,
                gradient_clip_val=CLIP,
                progress_bar_refresh_rate=1,
                log_every_n_steps=3,
                )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [9]:
trainer.fit(model, train_iter, dev_iter)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


['<extra_id_0>. <extra_id_10>', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.) <extra_id_37> с двигателем внутреннего сгорания', '<extra_id_0>.', '<extra_id_0>. <extra_id_10>. <extra_id_10>', '<extra_id_0>.', '<extra_id_0> Швейцарии']
['Чем заболел Байрон в Миссолонги?', 'Как отводятся излишки тепла у млекопитающих?', 'Что нарушают хромосомные аберрации?', 'В каком городе первый в мире городской автобус с двигателем внутреннего сгорания вышел на маршрут?', 'В каком режиме проходят подлимитные операции?', 'Находит ли отражение в алфавите фонетическая связь слогов?', 'Каким стал стиль последних работ художника?', 'Сколько человек родилось в 2008 году у граждан Швейцарии?']
['<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.', '<extra_id_0>.']
['Какая статья и пункт дает возможность статья для легализации ПО, скачанного из Интернет и предоставл', 'Какими русскими купцами был накоплен значимый торговый опыт?', 'Где была п

Global seed set to 294


In [ ]:
# from comet_ml import Experiment
# name = 't5_QG'
# trainer.save_checkpoint(f"/content/save_models/{name}.ckpt")
# experiment = Experiment(
#     api_key='HWfJT3eyByVJWe4nEbi1pGosA', project_name='qa',workspace='danildmitriev1999')

# experiment.log_model("t5_QG", f"/content/save_models/{name}.ckpt")

In [27]:
def generate_question(context, tokenizer, model):
    tokens = tokenizer.encode_plus(context,
                                    add_special_tokens=True,
                                    padding='max_length',
                                    max_length=350,
                                    truncation=True,
                                    return_tensors="pt")
    with torch.no_grad():
        generated_ids = model.generate(
                tokens["input_ids"],
                attention_mask=tokens["attention_mask"],
                use_cache=True,
                max_length=100, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
            )
    
    pred = [tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) 
    for generated_id in generated_ids]

    return pred

In [11]:
tr_model = model.model

In [44]:
dev_data[8]

{'context': 'Иногда говорят о возможности провести GNU GPL как договор присоединения, согласно статьям (428, 435 ГК РФ). Но единственный такой способ для лицензионных договоров описан в п. 3 ст. 1286 ГК РФ ( Заключение лицензионных договоров о предоставлении права использования программы для ЭВМ или базы данных допускается путём заключения каждым пользователем с соответствующим правообладателем договора присоединения, условия которого изложены на приобретаемом экземпляре таких программы или базы данных либо на упаковке этого экземпляра, а также в электронном виде (пункт 2 статьи 434). ). Эта статья даёт возможность для легализации ПО, скачанного из Интернет и предоставляемого по лицензии GNU GPL (Лицензионный договор, заключаемый в упрощённом порядке, является безвозмездным, если договором не предусмотрено иное.).',
 'id': '13772',
 'qas': [{'answers': [{'answer_start': 558, 'text': 'пункт 2 статьи 434'}],
   'id': '60110',
   'question': 'Какая статья и пункт дает возможность статья д

In [50]:
generate_question(dev_data[8]['context'], tokenizer, tr_model)

['Что является основой лицензии GNU GPL?']